In [23]:
import numpy as np
import pandas as pd

# Setting up and preprocessing data

In [41]:
n = np.NaN  # setting non scores to np.Nan for better integration with numpy
rows_users_ = ['u1', 'u2', 'u3', 'u4', 'u5', 'u6', 'u7']
columns_series = ['Vikings', 'Breaking Bad', 'The Sopranos', 'Westworld',
                  'Game of Thrones', 'The Witcher', 'Skam', 'Les bureau']
all_users = np.array([
    [5, 4, 1, n, 3, 4, 1],
    [5, 3, 3, 3, 4, 5, 1],
    [n, 3, 3, n, n, 2, n],
    [3, 4, 2, n, 4, 5, n],
    [4, 5, 2, n, 5, 3, 2],
    [n, 2, 3, 5, 5, 2, 1],
    [n, 3, 4, n, 1, 2, n],
    [n, 3, 1, n, n, 3, n],
])
all_users = all_users.T  # transform to get users as rows and series as columns
df = pd.DataFrame(all_users, columns=columns_series, index=rows_users_)
df

,Vikings,Breaking Bad,The Sopranos,Westworld,Game of Thrones,The Witcher,Skam,Les bureau
u1,5.0,5.0,NaN,3.0,4.0,NaN,NaN,NaN
u2,4.0,3.0,3.0,4.0,5.0,2.0,3.0,3.0
u3,1.0,3.0,3.0,2.0,2.0,3.0,4.0,1.0
u4,NaN,3.0,NaN,NaN,NaN,5.0,NaN,NaN
u5,3.0,4.0,NaN,4.0,5.0,5.0,1.0,NaN
u6,4.0,5.0,2.0,5.0,3.0,2.0,2.0,3.0
u7,1.0,1.0,NaN,NaN,2.0,1.0,NaN,NaN


In [42]:
# calculating means for all users
users_mean = np.nanmean(all_users, axis=0).reshape(1, 8)
df = pd.DataFrame(users_mean)
df

,0,1,2,3,4,5,6,7
0,3.0,3.428571,2.666667,3.6,3.5,3.0,2.5,2.333333


In [47]:
# calculating variance for all users
all_users_variance = np.empty(all_users.shape)
for i, user_scores in enumerate(all_users):
    all_users_variance[i] = user_scores - users_mean[0, i]
df = pd.DataFrame(all_users_variance, columns=columns_series, index=rows_users_)
df

,Vikings,Breaking Bad,The Sopranos,Westworld,Game of Thrones,The Witcher,Skam,Les bureau
u1,2.000000,2.000000,NaN,0.000000,1.000000,NaN,NaN,NaN
u2,0.571429,-0.428571,-0.428571,0.571429,1.571429,-1.428571,-0.428571,-0.428571
u3,-1.666667,0.333333,0.333333,-0.666667,-0.666667,0.333333,1.333333,-1.666667
u4,NaN,-0.600000,NaN,NaN,NaN,1.400000,NaN,NaN
u5,-0.500000,0.500000,NaN,0.500000,1.500000,1.500000,-2.500000,NaN
u6,1.000000,2.000000,-1.000000,2.000000,0.000000,-1.000000,-1.000000,0.000000
u7,-1.500000,-1.500000,NaN,NaN,-0.500000,-1.500000,NaN,NaN


In [48]:
# initializing active user
active_user = np.array([3, 2, 3, 0, 4, 1, 0, 5])
active_user

array([3, 2, 3, 0, 4, 1, 0, 5])

In [49]:
# calculating mean for active user
active_user_mean = np.nanmean(active_user)
active_user_mean

2.25

In [64]:
# calculating variance for active user
active_users_variance = np.zeros((1, 8))
for i, score in enumerate(active_user):
    active_users_variance[0, i] = score - active_user_mean
df = pd.DataFrame(active_users_variance)
df

,0,1,2,3,4,5,6,7
0,0.75,-0.25,0.75,-2.25,1.75,-1.25,-2.25,2.75


# Solving assignment
- Estimate the score this person is likely to give Westworld and Skam
- Solve the problem using Python and the corrcoef-function from Numpy

In [59]:
# calculating correlations between active user and all users
correlations = []
for other_users_variance in all_users_variance:
    temp_active = []
    temp_other = []
    for i in range(len(active_users_variance[0])):
        if not np.isnan(active_users_variance[0, i]) and not np.isnan(other_users_variance[i]):
            temp_active.append(active_users_variance[0, i])
            temp_other.append(other_users_variance[i])
    correlations_matrix = np.corrcoef(np.array(temp_active), np.array(temp_other))
    correlation = correlations_matrix[0][1]
    correlations.append(correlation)
df = pd.DataFrame(correlations)
df

,0
0,0.560612
1,0.276596
2,-0.643204
3,-1.000000
4,0.433861
5,-0.091245
6,0.774597


In [61]:
# function for estimating score of series for active user
def estimate_score(kappa,average_vote_for_a, product_index):
    sum_correlation_variance = 0
    j = product_index  # index of product we want to predict the vote for
    for i in range(len(all_users_variance)):
        if not np.isnan(all_users_variance[i][j]):
            sum_correlation_variance += all_users_variance[i][j] * correlations[i]
    p_a_j = average_vote_for_a + kappa * sum_correlation_variance
    return p_a_j

In [62]:
westworld = 3
estimate_score(1,active_user_mean,westworld)

2.8712982191008702

In [63]:
skam = 6
estimate_score(1,active_user_mean,skam)

0.280445998297429